In [332]:
import pandas as pd
from collections import Counter
import numpy as np

df = pd.read_csv(
               "fake_data.csv",
                 encoding_errors="ignore", 
                 on_bad_lines='skip', 
                 sep=",",
              
                 )

In [333]:
df["TIMESTAMP"] = pd.to_datetime(df["TIMESTAMP"], infer_datetime_format=True)
df = df.sort_values(["SessionID", "TIMESTAMP"])
table_1 = df
print("Table 1")
table_1

Table 1


,SessionID,CustomerID,Activity,TIMESTAMP
0,Session1,1,Start_application,2015-11-06 08:07:22.780
1,Session1,1,Input_info,2015-11-06 08:07:40.767
2,Session1,1,Send_application,2015-11-06 08:07:51.390
3,Session1,1,Accept_offer,2015-11-06 08:08:06.003
4,Session2,101,Send_application,2016-02-28 08:17:15.947
5,Session2,101,Accept_offer,2016-02-28 08:18:31.454
6,Session3,224,Input_info,2016-01-14 08:32:11.511
7,Session3,224,Send_application,2016-01-14 08:34:12.123
8,Session3,224,Accept_offer,2016-01-14 08:37:23.984
9,Session4,7653,Start_application,2016-02-20 20:15:10.321


In [334]:
#Create consecutive column with the consecutive activity and count how many time each pair occurs
df["Consecutive_activity"] = df.groupby("SessionID")["Activity"].shift(periods=-1)

#Count all the pairs and safe in new DataFrame
df_for_pivot_1 = pd.DataFrame(Counter(list(zip(df['Activity'], df['Consecutive_activity'],))), index=["value"]).T.reset_index()

df_for_pivot_1.fillna("end_session", inplace=True)

df_for_pivot_1.rename(columns = {"level_0":"Activity","level_1":"Consecutive activity","value":"Count"},inplace=True)
table_3 = df_for_pivot_1


In [335]:
table_2 = df.loc[:,("SessionID","Activity","Consecutive_activity")]
table_2.fillna("end_session", inplace=True)
print("Table 2")
table_2

Table 2


,SessionID,Activity,Consecutive_activity
0,Session1,Start_application,Input_info
1,Session1,Input_info,Send_application
2,Session1,Send_application,Accept_offer
3,Session1,Accept_offer,end_session
4,Session2,Send_application,Accept_offer
5,Session2,Accept_offer,end_session
6,Session3,Input_info,Send_application
7,Session3,Send_application,Accept_offer
8,Session3,Accept_offer,end_session
9,Session4,Start_application,Input_info


In [336]:
table_3.sort_values("Activity")

,Activity,Consecutive activity,Count
3,Accept_offer,end_session,4
1,Input_info,Send_application,2
4,Input_info,end_session,1
2,Send_application,Accept_offer,3
0,Start_application,Input_info,2


5.1.2

In [337]:
def make_pivot(df, index_names, column_names):
    """
    Functions that takes in a DataFrame and returns a pivot table with all the chances
    
    Create pivot table where chances are calculated that each row is succeded by activity that is represented in the column
    """
    df_chances = df.pivot_table(index=index_names, columns=column_names, values='Count')
    
    #Calculate what the probabilities are by summing the row and dividing all the values in the row by total sum of the row
    df_chances["total_row_count"] = df_chances.sum(axis=1)
    
    # tabel 4
    df_chances = df_chances.div(df_chances["total_row_count"], axis=0)
    df_chances["Start_application"] = 0
    df_chances.fillna(0, inplace=True)
    
    df_chances.drop("total_row_count", axis=1, inplace=True)
    df_chances = df_chances[["Accept_offer","Input_info","Send_application","Start_application","end_session"]]
    

    return df_chances

In [338]:
table_5 = make_pivot(table_3, "Activity","Consecutive activity")
table_5

Consecutive activity,Accept_offer,Input_info,Send_application,Start_application,end_session
Activity,,,,,
Accept_offer,0.0,0.0,0.000000,0,1.000000
Input_info,0.0,0.0,0.666667,0,0.333333
Send_application,1.0,0.0,0.000000,0,0.000000
Start_application,0.0,1.0,0.000000,0,0.000000


In [339]:
table_6 = table_5.copy()

#Calculate the probability of each activity that it is the first activity performed in the session
table_6["start_session_chance"] = df.groupby("SessionID").nth(0)["Activity"].value_counts() / df.groupby("SessionID").nth(0)["Activity"].value_counts().sum()
table_6["start frequency"] = df.groupby("SessionID").nth(0)["Activity"].value_counts()
table_6.fillna(0, inplace=True)

#Split the start probabilities from the normal DataFrame
start_chances = table_6.loc[:,("start_session_chance","start frequency")]


#Create df with all chances except the starting chance.
df_chances_rest = table_6.iloc[:, :-1]

sum_start_chances = start_chances['start_session_chance'].sum()
sum_start_frequency = start_chances['start frequency'].sum()
print("Table 6")
# start_chances.loc[len(start_chances)] = [sum_start_chances,sum_start_frequency]
Sum = start_chances.sum(axis=0)
start_chances.loc[len(start_chances)] = Sum
start_chances = start_chances[["start frequency","start_session_chance"]]
table_6 = start_chances
table_6

Table 6


Consecutive activity,start frequency,start_session_chance
Activity,,
Accept_offer,1.0,0.2
Input_info,1.0,0.2
Send_application,1.0,0.2
Start_application,2.0,0.4
4,5.0,1.0


In [340]:
table_5

Consecutive activity,Accept_offer,Input_info,Send_application,Start_application,end_session,start_session_chance,start frequency
Activity,,,,,,,
Accept_offer,0.0,0.0,0.000000,0,1.000000,0.2,1
Input_info,0.0,0.0,0.666667,0,0.333333,0.2,1
Send_application,1.0,0.0,0.000000,0,0.000000,0.2,1
Start_application,0.0,1.0,0.000000,0,0.000000,0.4,2


In [341]:
table_4 = table_3.pivot_table(index="Activity", columns="Consecutive activity", values='Count')
table_4.fillna(0,inplace=True)
table_4["Start_application"] = 0 
table_4["Sum"] = table_4.sum(axis=1)
table_4 = table_4[["Accept_offer","Input_info","Send_application","Start_application","end_session","Sum"]]
print("Table 4")
table_4

Table 4


Consecutive activity,Accept_offer,Input_info,Send_application,Start_application,end_session,Sum
Activity,,,,,,
Accept_offer,0.0,0.0,0.0,0,4.0,4.0
Input_info,0.0,0.0,2.0,0,1.0,3.0
Send_application,3.0,0.0,0.0,0,0.0,3.0
Start_application,0.0,2.0,0.0,0,0.0,2.0


In [342]:
df_chances_1 = make_pivot(df_for_pivot_1, "Activity", "Consecutive activity")

#Calculate the probability of each activity that it is the first activity performed in the session
df_chances_1["start_session_chance"] = df.groupby("SessionID").nth(0)["Activity"].value_counts() / df.groupby("SessionID").nth(0)["Activity"].value_counts().sum()
df_chances_1["start frequency"] = df.groupby("SessionID").nth(0)["Activity"].value_counts()
df_chances_1.fillna(0, inplace=True)

table_5 = df_chances_1
print("Table 5")
table_5.loc[:,("Accept_offer","Input_info","Send_application","end_session","start_session_chance","start frequency")]

Table 5


Consecutive activity,Accept_offer,Input_info,Send_application,end_session,start_session_chance,start frequency
Activity,,,,,,
Accept_offer,0.0,0.0,0.000000,1.000000,0.2,1
Input_info,0.0,0.0,0.666667,0.333333,0.2,1
Send_application,1.0,0.0,0.000000,0.000000,0.2,1
Start_application,0.0,1.0,0.000000,0.000000,0.4,2


In [343]:
df_chances_total = df_chances_1 

#Split the start probabilities from the normal DataFrame
start_chances = df_chances_total.loc[:,("start_session_chance","start frequency")]


#Create df with all chances except the starting chance.
df_chances_rest = df_chances_total.iloc[:, :-1]

sum_start_chances = start_chances['start_session_chance'].sum()
sum_start_frequency = start_chances['start frequency'].sum()
print("Table 6")
# start_chances.loc[len(start_chances)] = [sum_start_chances,sum_start_frequency]
Sum = start_chances.sum(axis=0)
start_chances.loc[len(start_chances)] = Sum
start_chances


Table 6


Consecutive activity,start_session_chance,start frequency
Activity,,
Accept_offer,0.2,1.0
Input_info,0.2,1.0
Send_application,0.2,1.0
Start_application,0.4,2.0
4,1.0,5.0


In [344]:
table_4

Consecutive activity,Accept_offer,Input_info,Send_application,Start_application,end_session,Sum
Activity,,,,,,
Accept_offer,0.0,0.0,0.0,0,4.0,4.0
Input_info,0.0,0.0,2.0,0,1.0,3.0
Send_application,3.0,0.0,0.0,0,0.0,3.0
Start_application,0.0,2.0,0.0,0,0.0,2.0
